# Create Hardiness Zones

Based on the hardines zones function in the xclim library: https://xclim.readthedocs.io/en/stable/indices.html#xclim.indices.hardiness_zones

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_path = '../data/processed/df_easternmountain_weather.parquet.gzip'
df = pd.read_parquet(df_path)
df['time']=pd.to_datetime(df['time'])
df.head()

,lat,lon,time,fahrenheit
0,35.25,-83.75,1985-01-01,51.242004
1,35.25,-83.50,1985-01-01,51.422016
2,35.25,-83.25,1985-01-01,50.450001
3,35.25,-83.00,1985-01-01,50.036034
4,35.25,-82.75,1985-01-01,51.296001


In [3]:
def temp_min(df,freq='YS'):
    ''' Takes DataFrame with temperature data and finds minimum temperature in the frequency given'''
    if df.index.name=='time':
        return df.groupby([pd.Grouper(level='time',freq=freq),'lat','lon']).min()
    else:
        return df.groupby([pd.Grouper(key='time',freq=freq),'lat','lon']).min()

In [4]:
small_df = df.iloc[:5000]
min_df = temp_min(small_df)
min_df.head()

fahrenheit
time       lat   lon               
1985-01-01 35.25 -83.75   -0.075985
                 -83.50    0.176014
                 -83.25    3.326014
                 -83.00   10.364019
                 -82.75   12.056025

In [5]:
#rolling window
def extract_hardiness(df,window=30):
    '''Get USDA Hardiness Zones with rolling window'''
    window_df = df.rolling(window).mean()
    binned = pd.cut(window_df['fahrenheit'],bins=range(-60,70,5),labels=np.linspace(1,13,25))
    return binned

In [6]:
extract_hardiness(min_df)

time        lat    lon   
1985-01-01  35.25  -83.75    NaN
                   -83.50    NaN
                   -83.25    NaN
                   -83.00    NaN
                   -82.75    NaN
                            ... 
            39.50  -78.75    7.5
                   -78.50    7.5
                   -78.25    7.5
                   -78.00    7.5
                   -77.75    7.5
Name: fahrenheit, Length: 279, dtype: category
Categories (25, float64): [1.0 < 1.5 < 2.0 < 2.5 ... 11.5 < 12.0 < 12.5 < 13.0]